In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Data Preprocessing

In [ ]:
#Import the dataset
data = pd.read_csv("/kaggle/input/churn-modelling/Churn_Modelling.csv")
data.head(5)

In [ ]:
#Load x and y into an array
x = Data.iloc[:,3:-1].values
y = Data.iloc[:,-1].values

#LabelEncoder to encode Gender to 0 and 1 that the model can understand
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2] )

#OneHotEncoder to encode Geography column to numbers that the model can understand
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers= [ ("encoder", OneHotEncoder(),[1])], remainder="passthrough")
x = np.array(ct.fit_transform(x))

#Train Test Split

from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size =0.2, random_state=88, shuffle = True ,
                                                    stratify = y )



Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train )
x_test = sc.transform(x_test )

x_train[0:6,:]

Building the ANN

In [ ]:
#Initializing the ANN
ann = tns.keras.models.Sequential() 

#Add the input layer and first hidden layer
ann.add(tns.keras.layers.Dense(units=6, activation = "relu")) 

#add a second hidden layer
ann.add(tns.keras.layers.Dense(units=6, activation = "relu")) 

#Add an output layer
ann.add(tns.keras.layers.Dense(units=1, activation = "sigmoid")) 

Training the ANN

In [ ]:
#Compiling the ANN

ann.compile(optimizer = "adam", loss= "binary_crossentropy" , metrics = ["accuracy"] ) 

#Training the ANN on the Training set

ann.fit(x_train, y_train, batch_size= 32 , epochs= 100 ) #default 32 is batch_size

Making the predictions and evaluating the model

In [ ]:
#Predict with test data

y_pred = ann.predict(x_test)


#Converts the probability to actual yes or no or 1 or 0
actual_pred = []
for i in y_pred:
    if i > 0.5:
        actual_pred.append( 1 )
    else:
        actual_pred.append( 0)
 

y_pred = np.array(actual_pred)


pred_actual = np.concatenate((y_pred.reshape(-1,1), y_test.reshape(-1,1)) , 1) 


#confusion matrix & accuracy score 

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score

cm = confusion_matrix(y_test,y_pred)
a_score = accuracy_score(y_test,y_pred)
r_score = recall_score(y_test,y_pred)
p_score = precision_score(y_test, y_pred)

cm


In [ ]:
a_score

In [ ]:
p_score

**The churn prediction ANN classification model achieved an accuracy score of 86% and a precision score of 74%.**